In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
# 명시적 대기를 위해 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time


In [2]:
# 상품 정보를 담는 클레스
class TourInfo:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    area  = ''
    link  = ''
    img   = ''
    contents = ''
    # 생성자
    def __init__(self, title, price, area, link, img, contents=None ):
        self.title = title
        self.price = price
        self.area  = area
        self.link  = link
        self.img   = img
        self.contents = contents

In [3]:
# 사전에 필요한 정보를 로드 => 디비혹스 쉘, 베치 파일에서 인자로 받아서 세팅

main_url = 'http://www.naver.com/' 
keyword  = '날씨'
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

In [4]:
# 윈도우용
driver = wd.Chrome(executable_path='.\\webdriver\\chromedriver.exe')

In [5]:
# 사이트 접속 (get)
driver.get(main_url)

In [6]:
# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('query').send_keys(keyword)

In [7]:
# 수정할경우 => 뒤에 내용이 붙어버림 => .clear() -> send_keys('내용')
# 검색 버튼 클릭
driver.find_element_by_id('search_btn').click()

In [8]:
time.sleep(5)

In [9]:
driver.close()

In [6]:
'''
# 사이트 접속 (get)
driver.get(main_url)
# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
# 수정할경우 => 뒤에 내용이 붙어버림 => .clear() -> send_keys('내용')
# 검색 옆에 삼각형을 눌러주는 역할을 함
driver.find_element_by_css_selector('button.recent-btn').click()
'''

"\n# 사이트 접속 (get)\ndriver.get(main_url)\n# 검색창을 찾아서 검색어 입력\n# id : SearchGNBText\ndriver.find_element_by_id('SearchGNBText').send_keys(keyword)\n# 수정할경우 => 뒤에 내용이 붙어버림 => .clear() -> send_keys('내용')\n# 검색 옆에 삼각형을 눌러주는 역할을 함\ndriver.find_element_by_css_selector('button.recent-btn').click()\n"

In [7]:
# 잠시 대기 => 페이가 로드되고 나서 즉각적으로 데이터를 획득 하는 행위는 
# 명시적 대기 => 특정 요소가 로케이트(발결된때까지) 대기
try:
    element = WebDriverWait(driver, 10).until(
        # 지정한 한개 요소가 올라면 웨이트 종료
        EC.presence_of_element_located( (By.CLASS_NAME, 'oTravelBox') )
    )
except Exception as e:
    print( '오류 발생', e)
# 암묵적 대기 => DOM이 다 로드 될때까지 대기 하고 먼저 로드되면 바로 진행
# 요소를 찾을 특정 시간 동안 DOM 풀링을 지시 예를 들어 10 초이내 라로 
# 발견 되면 진행
driver.implicitly_wait( 10 )
# 절대기 대기 => time.sleep(10) -> 클라우드 페어(디도스 방어  솔류션)
# 더보기 눌러서 => 게시판 진입 
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

In [8]:
# searchModule.SetCategoryList(1, '') 스크립트 실행
# 16은 임시값, 게시물을 넘어갔을때 현상을 확인차
for page in range(1, 6): # 2):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        #############################################################
        # 여러 사이트에서 정보를 수집할 경우 공통 정보 정의 단계 필요
        # 상품명, 코멘트, 기간1, 기간2, 가격, 평점, 썸네일, 링크(상품상세정보)
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나 하나 접근
        for li in boxItems:
            # 이미지를 링크값을 사용할것인가? 
            # 직접 다운로드 해서 우리 서버에 업로드(ftp) 할것인가?
            print( '썸네임', li.find_element_by_css_selector('img').get_attribute('src') )
            print( '링크', li.find_element_by_css_selector('a').get_attribute('onclick') )
            print( '상품명', li.find_element_by_css_selector('h5.proTit').text )
            print( '코멘트', li.find_element_by_css_selector('.proSub').text )
            print( '가격',   li.find_element_by_css_selector('.proPrice').text )
            area = ''
            for info in li.find_elements_by_css_selector('.info-row .proInfo'):
                print(  info.text )
            print('='*100)
            # 데이터 모음
            # li.find_elements_by_css_selector('.info-row .proInfo')[1].text
            # 데이터가 부족하거나 없을수도 있으므로 직접 인덱스로 표현은 위험성이 있음
            obj = TourInfo(  
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('.proPrice').text,
                li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
                li.find_element_by_css_selector('a').get_attribute('onclick'),
                li.find_element_by_css_selector('img').get_attribute('src')
            )
            tour_list.append( obj )
    except Exception as e1:
        print( '오류', e1 )

print( tour_list, len(tour_list) )


1 페이지 이동
썸네임 http://tourimage.interpark.com/product/tour/00161/A10/280/A1033078_1_163.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A1033078','')
상품명 [롯데홈쇼핑★호텔특집전] 베트남or필리핀or태국(푸켓)or괌사이판 프리미엄호텔 3박 숙박권
코멘트 얼린 숙박권! 지금이 기회! 1년 내내 동일 가격!
가격 299,000 원~
여행 기간 : 3박4일
출발 가능 기간 : 2021.12.31~2023.12.31
평점 0
0개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A10/280/A1033078_1_163.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A1033078','')
상품명 [롯데홈쇼핑★호텔특집전] 베트남or필리핀or태국(푸켓)or괌사이판 프리미엄호텔 3박 숙박권
코멘트 얼린 숙박권! 지금이 기회! 1년 내내 동일 가격!
가격 299,000 원~
여행 기간 : 3박4일
출발 가능 기간 : 2021.12.31~2023.12.31
평점 0
0개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/B40/280/B4010929_10_623.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=B4010929','')
상품명 [괌 자유 4일/5일] PIC+골드카드+환타지디너쇼+자유여행
코멘트 최대워터파크&부대시설
가격 928,900 원~
여행 기간 : 4박5일
출발 가능 기간 : 2021.06.29~2022.03.26
평점 9.3
85개의 상품평
썸네임 

In [9]:
driver.close()